In [86]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np

true = np.asarray([[3, 2, 1, 0, 0]])
relevance = np.asarray([[3, 2, 0, 0, 1]])

print(ndcg_score(true, relevance))

#0.980840401274087

0.980840401274087


Задание 4.5

1 point possible (graded)

Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

Реальные оценки: [2, 4, 1, 1, 1]

Предсказанные оценки: [2, 5, 2, 3, 1]

Вычислите коэффициент NDCG. Округлите результат до двух знаков после точки-разделителя.

In [87]:
true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])

print(round(ndcg_score(true, relevance),2))

0.97


In [88]:
import pandas as pd

In [89]:
movies = pd.read_csv("movie.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [90]:
ratings = pd.read_csv("dst-3.0_mathml_14_5_rating.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [91]:
df=pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Задание 5.1


Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.


In [92]:
movie_popularity = df["title"].value_counts()
movie_popularity

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
                                    ...  
Richard III (1912)                      1
The Nautical Chart (2007)               1
Two Arabian Knights (1927)              1
Night Flight (1933)                     1
Série noire (1979)                      1
Name: count, Length: 26729, dtype: int64

Задание 5.2

Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных Movie Lens есть оценки каждого из зрителей.

Найдите средний рейтинг для каждого из фильмов.
Найдите фильмы с наивысшим средним рейтингом.
Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.
Впишите только название фильма, без кавычек и без года выхода на экран.

Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

In [93]:
average_rating_df = df[["title", "rating"]].groupby('title').mean()

sorted_average_ratings = average_rating_df.sort_values(by=["rating",'title'], ascending=False)

print(sorted_average_ratings.head(10))

                                                   rating
title                                                    
Yonkers Joe (2008)                                    5.0
Year Zero: The Silent Death of Cambodia (1979)        5.0
Who Killed Vincent Chin? (1987)                       5.0
When I Walk (2013)                                    5.0
Welcome to Australia (1999)                           5.0
Victor and the Secret of Crocodile Mansion (2012)     5.0
Turkish Dance, Ella Lola (1898)                       5.0
This Thing With Sarah (2013)                          5.0
The great match (2007)                                5.0
The Wrecking Crew (2008)                              5.0


Задание 5.3

В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

Сколько таких фильмов?

In [94]:
movie_popularity = df["title"].value_counts()
movie_popularity

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
                                    ...  
Richard III (1912)                      1
The Nautical Chart (2007)               1
Two Arabian Knights (1927)              1
Night Flight (1933)                     1
Série noire (1979)                      1
Name: count, Length: 26729, dtype: int64

In [95]:
popular_movies = movie_popularity[movie_popularity > 50].index

In [96]:
print(popular_movies.shape[0])

10472


Задание 5.4

Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей средней оценкой. В качестве ответа запишите название этого фильма без артикля и года выхода на экран.
Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

In [97]:
movie_popularity = df["title"].value_counts()
popular_movies = movie_popularity[movie_popularity > 50].index
popular_movies

Index(['Pulp Fiction (1994)', 'Forrest Gump (1994)',
       'Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)',
       'Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Braveheart (1995)', 'Terminator 2: Judgment Day (1991)',
       'Matrix, The (1999)', 'Schindler's List (1993)',
       ...
       'Light Years (Gandahar) (1988)', 'Black Water (2007)',
       'Salome's Last Dance (1988)',
       'Flower of Evil, The (Fleur du mal, La) (2003)',
       'California Dreamin' (Nesfarsit) (2007)',
       'Serpent's Egg, The (Schlangenei, Das) (1977)',
       'Lion of the Desert (1980)', 'Lakeboat (2000)', 'Mad Love (1935)',
       'Kid Brother, The (1927)'],
      dtype='object', name='title', length=10472)

In [98]:
df2 = df[df.title.isin(popular_movies)]
average_rating_df2 = df2[["title", "rating"]].groupby('title').mean()

sorted_average_ratings = average_rating_df2.sort_values(by=["rating",'title'], ascending=False)
sorted_average_ratings

,rating
title,
"Shawshank Redemption, The (1994)",4.446990
"Godfather, The (1972)",4.364732
"Usual Suspects, The (1995)",4.334372
Schindler's List (1993),4.310175
"Godfather: Part II, The (1974)",4.275641
...,...
Barney's Great Adventure (1998),1.163484
Glitter (2001),1.124088
Bratz: The Movie (2007),1.105556


In [99]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gspmoreira/articles-sharing-reading-from-cit-deskdrop")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\molde\.cache\kagglehub\datasets\gspmoreira\articles-sharing-reading-from-cit-deskdrop\versions\5


shared_articles

In [100]:
shared_articles = pd.read_csv("shared_articles.csv")
shared_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


Задание 6.1


Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [101]:
shared_articles[shared_articles['eventType'] == 'CONTENT SHARED'].shape[0]

3047

In [102]:
users_interactions = pd.read_csv("users_interactions.csv")
users_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [103]:
users_interactions.personId = users_interactions.personId.astype(str)
users_interactions.contentId = users_interactions.contentId.astype(str)
shared_articles.contentId = shared_articles.contentId.astype(str)

Задание 6.2

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [104]:
def clss(x):
  if x == 'VIEW':
    return 1.0
  if x == 'LIKE':
    return 2.0 
  if x == 'BOOKMARK':
    return 2.5
  if x == 'FOLLOW':
    return 3.0
  if x == 'COMMENT CREATED':
    return 4.0
  else:
    return 0

In [105]:
users_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    72312 non-null  int64 
 1   eventType    72312 non-null  object
 2   contentId    72312 non-null  object
 3   personId     72312 non-null  object
 4   sessionId    72312 non-null  int64 
 5   userAgent    56918 non-null  object
 6   userRegion   56907 non-null  object
 7   userCountry  56918 non-null  object
dtypes: int64(2), object(6)
memory usage: 4.4+ MB


In [106]:
users_interactions['coded'] = users_interactions['eventType'].apply(clss)
round(users_interactions['coded'].mean(),2)

1.24

In [107]:
users_interactions

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,coded
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


Задание 6.3

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?


In [108]:
movie_popularity = df["title"].value_counts()
popular_movies = movie_popularity[movie_popularity > 50].index
popular_movies

Index(['Pulp Fiction (1994)', 'Forrest Gump (1994)',
       'Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)',
       'Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Braveheart (1995)', 'Terminator 2: Judgment Day (1991)',
       'Matrix, The (1999)', 'Schindler's List (1993)',
       ...
       'Light Years (Gandahar) (1988)', 'Black Water (2007)',
       'Salome's Last Dance (1988)',
       'Flower of Evil, The (Fleur du mal, La) (2003)',
       'California Dreamin' (Nesfarsit) (2007)',
       'Serpent's Egg, The (Schlangenei, Das) (1977)',
       'Lion of the Desert (1980)', 'Lakeboat (2000)', 'Mad Love (1935)',
       'Kid Brother, The (1927)'],
      dtype='object', name='title', length=10472)

In [109]:
users_interactions_count_df = (
    users_interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print(len(users_with_enough_interactions_df))

1140


Задание 6.4

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

В этом задании необходимо работать с данными, полученными в задании 6.3.



In [117]:
users_with_enough_interactions_df

,personId
0,-1007001694607905623
1,-1032019229384696495
2,-108842214936804958
3,-1119397949556155765
4,-1130272294246983140
...,...
1135,953707509720613429
1136,983095443598229476
1137,989049974880576288
1138,997469202936578234


In [121]:
interactions_from_selected_users_df = users_interactions.loc[np.in1d(users_interactions.personId,users_with_enough_interactions_df)]
print(interactions_from_selected_users_df.shape)

(69868, 9)


In [120]:
interactions_from_selected_users_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,coded
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

def smooth_user_preference(x):
    return math.log(1+x, 2)

Задание 6.5

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

В этом задании необходимо работать с данными, полученными в задании 6.4.

In [138]:
interactions_from_selected_users_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,coded
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [134]:
import math

In [142]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'], as_index=False)['coded'].sum()
)
interactions_full_df['coded'] = interactions_full_df['coded'].apply(smooth_user_preference)

interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'], as_index=False)['timestamp'].max()
)['timestamp']
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df['last_timestamp'].mean()

1470605340.0403006